# Day 6 – Chain-of-Thought & ReAct Agents + LangGraph Primer  
### Project Proposal • Special Topics

| 🔍 Topic | 📚 What you’ll learn | 🛠️ Hands-on |
|----------|----------------------|-------------|
| Chain-of-Thought (CoT) | Prompt pattern for step-by-step reasoning | Build a zero-tool CoT agent |
| ReAct framework | Thought → Action → Observation loop | Revisit ReAct with a richer toolset |
| LangGraph deep dive | Turns your agent workflow into an explicit graph | Visualize & execute a CoT→Tool hybrid |
| Project proposals | Pitch capstone ideas | Break into teams & start skeleton repos |

> **Audience:** HS students, beginner → intermediate Python  
> **Prereqs:** Days 1-5 notebooks  
> **Resources:** OpenAI API key, internet access for `wikipedia` only


In [14]:
# %% ------------------------------------------------------------
# 📦 Install / Imports (run once)
# --------------------------------------------------------------
# !pip install langchain-openai langgraph wikipedia python-dotenv rich --quiet

import os
from typing import List, Dict, Any, TypedDict
from dotenv import load_dotenv
load_dotenv()                         # expects OPENAI_API_KEY in .env

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END, START
from pydantic import BaseModel, Field
from IPython.display import Markdown, Image

In [6]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
print("✅ LLM ready ✔︎")

✅ LLM ready ✔︎


## Step 1: What is *Chain-of-Thought* (CoT)?

* **Idea:** Ask the LLM to *think out loud* before giving the final answer.  
* **Benefits:**  
  * Higher accuracy on reasoning tasks  
  * Easier to debug – you see every thought  
* **Trade-off:**  
  * Verbose output  
  * Slightly higher token cost


In [7]:
# %% ------------------------------------------------------------
# 🧠 Implement a minimal CoT agent (no external tools)
# --------------------------------------------------------------
COT_SYSTEM_PROMPT = """
You are a helpful tutor who always thinks step-by-step.
When answering a question, first write out your chain-of-thought
under a header "Thoughts:", then give the short final answer
under "Answer:".
"""

def cot_agent(question: str) -> str:
    messages = [
        SystemMessage(COT_SYSTEM_PROMPT),
        HumanMessage(question)
    ]
    response = llm.invoke(messages)
    return response.content

# ▸ Quick smoke test
print(cot_agent("If I have 7 apples and give away 3, how many remain?"))

Thoughts: 
To find out how many apples remain after giving away some, I need to subtract the number of apples given away from the total number of apples I initially have. I start with 7 apples and give away 3. 

So, the calculation is:
7 (initial apples) - 3 (given away) = 4 (remaining apples).

Answer: 4 apples remain.


## 2 ▸ ReAct recap (Thought → *Action* → Observation → Answer)

ReAct extends CoT by letting the agent **call tools** inside its reasoning loop.

<div style="text-align:center;">
    <img src="../assets/react.png" width="50%" height="50%"/>
</div>

In [8]:
# %% ------------------------------------------------------------
# 🔧 Build a richer tool belt for today's ReAct demo
# --------------------------------------------------------------
@tool
def add(x: float, y: float) -> float:
    """Return x + y"""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Return x * y"""
    return x * y

import wikipedia
@tool
def wiki(topic: str, sentences: int = 2) -> str:
    """Wikipedia summary"""
    return wikipedia.summary(topic, sentences=sentences)

TOOLS = [add, multiply, wiki]

In [11]:
# %% ------------------------------------------------------------
# 🤖 Spin up a ReAct agent using LangGraph's helper
# --------------------------------------------------------------
react_agent = create_react_agent(
    llm,
    tools=TOOLS)

react_agent.invoke({"messages": "What is 8*12 and who designed the Eiffel Tower?"})

{'messages': [HumanMessage(content='What is 8*12 and who designed the Eiffel Tower?', additional_kwargs={}, response_metadata={}, id='f21b82da-6d77-43bb-97d2-6b8a5c336a65'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kdyttyJtNbzOUo0KH0PQEzKV', 'function': {'arguments': '{"x": 8, "y": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_kzwnDNPLx0zh6Ma2bZyUJmDt', 'function': {'arguments': '{"topic": "Eiffel Tower", "sentences": 2}', 'name': 'wiki'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 108, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d1666114-4db7-41c7-b374-